<h3>Tratar valores nulos nos dados de gatos utilizando média de outro dataset</h3>

In [1]:
import pandas as pd
dataset_dogs = pd.read_csv('../data/NYC_Dog_Licensing_Dataset.csv')

<b>Removendo colunas que não iremos utilizar</b>

In [2]:
dataset_dogs.drop(['RowNumber', 'CensusTract2010', 'NTA', 'CityCouncilDistrict', 'CommunityDistrict', 'CongressionalDistrict', 'StateSenatorialDistrict', 'LicenseIssuedDate', 'LicenseExpiredDate'], axis=1, inplace=True)
dataset_dogs.drop(dataset_dogs.index[67146:122203], axis=0, inplace=True)

In [3]:
from datetime import datetime

# substituindo os valores do campo AnimalBirthMonth para retirar a hora deixa somente a data
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'], format="%d/%m/%Y %H:%M:%S %p")
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'])
dataset_dogs['AnimalBirthMonth'] = dataset_dogs['AnimalBirthMonth'].map(lambda x: datetime.strftime(x,"%d/%m/%Y"), na_action='ignore')

In [4]:
dataset_dogs.to_csv('../data/dogs_clean.csv')

In [5]:
import os

dir_atual = os.getcwd()
dir_destino = os.path.abspath(os.path.join(dir_atual, os.pardir, '/home/yoseph/Documentos/sptech/tcc/grupo11-4cco-tcc/src/inteligencia_artificial/train/'))

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]
fotos = set(arquivos_jpg)

In [6]:
dataset_breed_dogs = pd.read_csv('../../data/labels.csv')

In [7]:
import base64

def is_base64(s):
    try:
        base64.b64decode(s)
        return True
    except Exception:
        return False

<h3>Adequação das raças de cachorros</h3>

In [8]:
breed_sample = dataset_breed_dogs.sample(n=len(dataset_dogs), replace=True)

# redefina o índice do DataFrame de raças
breed_sample = breed_sample.reset_index(drop=True)

# atualize a coluna 'BreedName' do DataFrame de cães com a coluna 'breed' do DataFrame de raças amostrado
dataset_dogs['BreedName'] = breed_sample['breed']

# salve o DataFrame atualizado em um novo arquivo CSV
dataset_dogs.to_csv('../../data/dogs_clean.csv', index=False)

In [9]:
dataset_dogs['BreedName'] = dataset_dogs['BreedName'].replace('Unknown', pd.NA)
breed_mean = dataset_dogs['BreedName'].dropna().mode()[0]
dataset_dogs['BreedName'] = dataset_dogs['BreedName'].fillna(breed_mean)

dataset_dogs.to_csv('../../data/dogs_clean.csv')

In [10]:
dataset_dogs['Image'] = None

In [11]:
dataset_dogs.head(3)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,Image
0,SHADOW,M,01/01/2000,eskimo_dog,Brooklyn,11236.0,None
1,ROCCO,M,10/01/2011,saluki,Brooklyn,11210.0,None
2,LUIGI,M,09/01/2005,tibetan_terrier,Bronx,10464.0,None


In [12]:
import base64
from PIL import Image
from tqdm import tqdm

def find_image_from_name_dog(breed_name):
    if breed_name == 'Unknown' or not breed_name:
        return None, dataset_breed_dogs
    breed_name_lower = str(breed_name).lower()
    breeds_founded = dataset_breed_dogs[dataset_breed_dogs['breed'].str.lower().str.contains(breed_name_lower, case=False, regex=False)]
    
    if breeds_founded.empty:
        return None, dataset_breed_dogs
    
    first_breed = breeds_founded.iloc[0]
    df = dataset_breed_dogs.drop(breeds_founded.index[0])
    info_breed = first_breed.to_dict()
    indice_arquivo = arquivos_jpg.index(f"{info_breed['id']}.jpg")
    nome_arquivo = arquivos_jpg[indice_arquivo]
    caminho_completo = os.path.join(dir_destino, nome_arquivo)
    img = Image.open(caminho_completo)
    img_bytes = img.tobytes()
    return base64.b64encode(img_bytes).decode('utf-8'), df

try:
    for index, row in tqdm(dataset_dogs.iterrows(), total=len(dataset_dogs)):
        image_base64, df = find_image_from_name_dog(row['BreedName'])
        dataset_breed_dogs = df
        if not is_base64(row["Image"]):
            dataset_dogs.loc[index, 'Image'] = image_base64
except Exception as error:
    print(error)

 11%|█         | 7224/67146 [02:53<26:48:12,  1.61s/it]

In [ ]:
dataset_breed_dogs.tail(2)

,id,breed


In [ ]:
dataset_dogs.head(10)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,Image
0,SHADOW,M,01/01/2000,japanese_spaniel,Brooklyn,11236.0,AAEAAwQAAQIABAUAAAEAAAEADhAPAAEAAAMEAAMEAAIBAA...
1,ROCCO,M,10/01/2011,dhole,Brooklyn,11210.0,WYlZTHxMP289Pm46SHdDUoFKVYRNU4NHYpBSYpBPY5BPYo...
2,LUIGI,M,09/01/2005,briard,Bronx,10464.0,aoNjaYJiaIFhZ4BgZ4BgaYJia4RkbYZma4Rka4Rka4RkbI...
3,PETUNIA,F,08/01/2013,schipperke,Brooklyn,11221.0,/////////////////////////////////f39/v7+/f39+v...
4,ROMEO,M,10/01/2008,old_english_sheepdog,Bronx,10451.0,IiATIiATIyEUJCIVJCIVIyEUIiATIiATJCIVJCIVJSMWJi...
5,BRANDY,M,01/01/2004,italian_greyhound,Brooklyn,11225.0,eAIAeAIAeAIAeAIAeQEAeQEAeQEAeQEAfAIBfAIBfQEBfQ...
6,SAM,M,05/01/2011,affenpinscher,Manhattan,10021.0,q3NOqnJNqHJMp3FLpnBKpnBKpnJLpnJLr35Ws4JatoVdto...
7,MAY,F,05/01/2004,kelpie,Staten Island,10305.0,4uTj5Obl5+no6+3s8PLx8/X09vj39/n4+fv6+fv6+vz7+/...
8,RUBY,F,04/01/2010,italian_greyhound,Brooklyn,11220.0,HyMkHyMkHyMkHyMkHyMkHyMkHyMkHyMkHSEiHSEiHSEiHS...
9,LEO,M,01/01/2014,norwich_terrier,Bronx,10468.0,CAMAEQoEEQQAIxMGEQAAJA4AHQkAGAYAEQAADgAAFgYGDw...


In [ ]:
dataset_dogs.to_csv('../../data/dogs_clean.csv')